In [2]:
%pip install -qU sentence-transformers
%pip install -q evaluate
%pip install torch --upgrade
%pip install accelerate --upgrade
%pip install transformers[torch] --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Мирославик\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Мирославик\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Мирославик\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Collecting accelerate
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/13/9e/ee987874058f2d93006961f6ff49e0bcb60ab9c26709ebe06bfa8707a4d8/accelerate-0.24.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/261.4 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/261.4 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/261.4 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/261.4 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/261.4 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/261.4 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/261.4 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/261.4 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/261.4 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/261.4 kB ? 


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Мирославик\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Мирославик\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import evaluate

from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score

C:\Users\Мирославик\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [5]:
train_data.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12120 entries, 0 to 12119
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          12120 non-null  object
 1   premise     12120 non-null  object
 2   hypothesis  12120 non-null  object
 3   lang_abv    12120 non-null  object
 4   language    12120 non-null  object
 5   label       12120 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 568.3+ KB


In [8]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5195 entries, 0 to 5194
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          5195 non-null   object
 1   premise     5195 non-null   object
 2   hypothesis  5195 non-null   object
 3   lang_abv    5195 non-null   object
 4   language    5195 non-null   object
dtypes: object(5)
memory usage: 203.1+ KB


In [9]:
train_data.label.value_counts()

label
0    4176
2    4064
1    3880
Name: count, dtype: int64

In [11]:
model_name = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'
tokenizer = AutoTokenizer.from_pretrained(model_name)

C:\Users\Мирославик\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Мирославик\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [12]:
rows = len(train_data.index)
n_train = int(int(rows*0.8))
important_labels = ['id','premise','hypothesis','label']



train_ds = train_data.loc[:n_train,important_labels]
train_ds = Dataset.from_pandas(train_ds)

val_ds = train_data.loc[n_train:,important_labels]
val_ds = Dataset.from_pandas(val_ds)

test_ds = Dataset.from_pandas(test_data)

ds = DatasetDict()

ds['train'] = train_ds
ds['validation'] = val_ds
ds['test'] = test_ds

In [13]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label'],
        num_rows: 9697
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label'],
        num_rows: 2424
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language'],
        num_rows: 5195
    })
})

In [14]:
def tokenizer_sentence(data):
    return tokenizer(data['premise'], data['hypothesis'], truncation=True)

In [15]:
tokenized_datasets = ds.map(tokenizer_sentence, batched=True)
tokenized_datasets

Map: 100%|██████████| 5195/5195 [00:00<00:00, 7227.16 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'input_ids', 'attention_mask'],
        num_rows: 9697
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2424
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'input_ids', 'attention_mask'],
        num_rows: 5195
    })
})

In [16]:
columns_to_remove = ["premise", "hypothesis"]
columns_to_remove_test = ["lang_abv", "language"]

tokenized_datasets = tokenized_datasets.remove_columns(columns_to_remove)
tokenized_datasets['test'] = tokenized_datasets['test'].remove_columns(columns_to_remove_test)

tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 9697
    })
    validation: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2424
    })
    test: Dataset({
        features: ['id', 'input_ids', 'attention_mask'],
        num_rows: 5195
    })
})

In [17]:
print(" ".join([str(i) for i in tokenized_datasets['train'][1]['input_ids']]))

0 32255 621 37348 450 642 148 56644 133 678 23 41361 94407 111 27165 11037 7 4 2412 2804 5 2 2 109613 13 94407 621 959 28897 3674 47 4488 98 6097 37348 5 2


In [18]:
print(" ".join([str(i) for i in tokenized_datasets['train'][1]['attention_mask']]))

1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
import torch.nn as nn
from transformers import XLMRobertaModel
import torch

class CustomXLMRobertaModel(nn.Module):
    def __init__(self, num_labels):
        model_name = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'
        super(CustomXLMRobertaModel, self).__init__()
        self.roberta = XLMRobertaModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Linear(768, num_labels)
        self.loss = nn.CrossEntropyLoss()
        self.num_labels = num_labels

        def forward(self, input_ids, attention_mask, labels=None):
            output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
            output = self.dropout(output.pooler_output)
            logits = self.classifier(output)

            if labels is not None:
                loss = self.loss(logits.view(-1, self.num_labels), labels.view(-1))
                return {"loss": loss, "logits": logits}
            else:
                return logits


num_labels = 3
model = CustomXLMRobertaModel(num_labels)

Some weights of XLMRobertaModel were not initialized from the model checkpoint at symanto/xlm-roberta-base-snli-mnli-anli-xnli and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
training_args = TrainingArguments("/content",
                                  optim="adamw_torch",
                                  num_train_epochs=3,
                                  evaluation_strategy="epoch",
                                  logging_dir='./logs',
                                  logging_steps=10)


In [22]:
def metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Расчёт метрик
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='micro')

    # Возвращаем словарь со всеми метриками
    return {
        'accuracy': accuracy,
        'f1': f1
    }

In [23]:
def compute_metrics(eval_preds):
    f1_metric = evaluate.load("f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return f1_metric.compute(predictions=predictions, references=labels, average="micro")

In [24]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=metrics,
)

In [25]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [26]:
trainer.train()

  0%|          | 0/3639 [00:00<?, ?it/s]You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']